In [12]:
import pandas as pd
import Bio, os, re
from Bio import AlignIO, SeqIO, pairwise2
from Bio.Seq import Seq 
import pydna
import subprocess
import pysam
from collections import defaultdict
from bs4 import BeautifulSoup

pd.options.display.max_rows = 1000

In [13]:
path = '/run/user/1701730025/gvfs/smb-share:server=s455fm120,share=share/DIS/LABS/Environmental/WNV/fasta_sequence/'
fasta_file = [path + x for x in os.listdir(path) if x.endswith("_sequence.fasta")]
fasta_file_name = [os.path.basename(x).replace("_sequence.fasta", "")  for x in fasta_file]

In [3]:
dicts = {}
fasta_dict = {}
for i in range(0, len(fasta_file)):
    dicts[fasta_file_name[i]] = len(SeqIO.read(fasta_file[i], "fasta").seq)
    fasta_dict[fasta_file_name[i]] = fasta_file[i]

In [4]:
#This is the 
class microbe:
    def __init__(self, forward, reverse, probe, fasta):
        self.forward = forward
        self.reverse = reverse
        self.probe = probe
        self.fasta = fasta
        
    def fr_position(self):
        if Seq(self.forward) in SeqIO.read(self.fasta, "fasta").seq:
            forward_position = SeqIO.read(self.fasta, "fasta").seq.index(Seq(self.forward))    
        elif Seq(self.forward).reverse_complement() in SeqIO.read(self.fasta, "fasta").seq:
            forward_position = SeqIO.read(self.fasta, "fasta").seq.index(Seq(self.forward).reverse_complement())
        else:
            forward_position = "no_forward"
            
        if Seq(self.reverse) in SeqIO.read(self.fasta, "fasta").seq:
            reverse_position = SeqIO.read(self.fasta, "fasta").seq.index(Seq(self.reverse))
        elif Seq(self.reverse).reverse_complement() in SeqIO.read(self.fasta, "fasta").seq:
            reverse_position = SeqIO.read(self.fasta, "fasta").seq.index(Seq(self.reverse).reverse_complement())
        else:
            reverse_position = "no_reverse"
        
        if type(forward_position) == int and type(reverse_position) == int:
            if reverse_position > forward_position:
                amplicon_lenght = reverse_position + len(Seq(self.reverse)) - forward_position + 1
            else:
                amplicon_lenght = forward_position + len(Seq(self.forward)) - reverse_position + 1
            return([forward_position, reverse_position, amplicon_lenght])
        else:
            return([forward_position, reverse_position])

In [5]:
Anaplasma_phagocytophilum = microbe("GGCATGTAGGCGGTTCGGT", "CACTAGGAATTCCGCTATCCTCTCC", "GCCAGGGCTTAACCCTGGAGCT",\
                                    fasta_dict['Anaplasma_phagocytophilum'])
Powassan_virus = microbe('GTGATGTGGCAGCGCACC', 'CTGCGTCGGGAGCGACCA', 'CCTACTGCGGCAGCACACACAGTG',\
                        fasta_dict['Powassan_virus'])
Borrelia_miyamotoi = microbe('AGCACAAGCTTCATGGACATTGA', 'GAGCTGCTTGAGCACCTTCTC', 'TGTGGGTGCAAATCAGGATGAAGCA',\
                            fasta_dict['Borrelia_miyamotoi'])
Borrelia_burgdorferi = microbe('CCTTCAAGTACTCCAGATCCATTG', 'AACAAAGACGGCAAGTACGATC', 'CAACAGTAGACAAGCTTGA',\
                              fasta_dict['Borrelia_burgdorferi'])
Babesia_microti = microbe('CATCATGCCAGGCCTGTTTG', 'GAAGAAACCACAAGAGCAAATGC', 'TACTACCCATACTGGTCGGTGCTCC',\
                         fasta_dict['Babesia_microti'])
Anaplasma_platys = microbe("GGCATGTAGGCGGTTCGGT", "CACTAGGAATTCCGCTATCCTCTCC", "GCCAGGGCTTAACCCTGGAGCT",\
                                    fasta_dict['Anaplasma_platys'])

In [6]:
microbe_list = [Anaplasma_phagocytophilum, Powassan_virus, Borrelia_miyamotoi, Borrelia_burgdorferi, Babesia_microti,
               Anaplasma_platys]

In [7]:
len_dict = {}
for m in range(0, len(microbe_list)):
    len_dict[fasta_file_name[m]] = microbe.fr_position(microbe_list[m])

In [8]:
len_dict 

{'Powassan_virus': [1065588, 1065497, 111],
 'Babesia_microti': [10384, 10547, 182],
 'Anaplasma_phagocytophilum': [757086, 757187, 123],
 'Anaplasma_platys': [9809, 9738, 96],
 'Borrelia_miyamotoi': [2765, 2880, 139],
 'Borrelia_burgdorferi': [489, 574, 111]}

In [9]:
def degenerate_base(base):
    if base == 'Y':
        return 'CT'
    elif base == 'R':
        return 'AG'
    elif base == 'W':
        return 'AT'
    elif base == 'S':
        return 'GC'
    elif base == 'K':
        return 'TG'
    elif base == 'M':
        return 'AC'
    elif base == 'D':
        return 'AGT'
    elif base == 'V':
        return 'ACG'
    elif base == 'H':
        return 'ACT'
    elif base == 'B':
        return 'CGT'
    elif base == 'X' or base == 'N':
        return 'ATGC'
    else:
        return base
    
def is_frp(frp, frp_key):
    frp_bool_list = []
    for i in frp_key:
        frp_bool_list.append(bool(re.match(frp, i, re.IGNORECASE)))
    return [i for (i, v) in zip(frp_key, frp_bool_list) if v][0]

def listfiles(path, extension):
    dirpath = [k for k in os.listdir(path) if extension in k]
    return dirpath

def align_summary(path):
    from Bio import AlignIO
    path = path

    dlist = []
    dlist2 = []

    for file in listfiles(path, "clustalw"):
        alignment = AlignIO.read(open(path + file), "clustal")

        align_dict = {}
        for i in range(0, len(alignment)):
            align_dict[alignment[i].id] = str(alignment[i].seq)

            frp_key = [x for x in align_dict.keys() if re.search("forward|reverse|probe", x, re.IGNORECASE) != None]
            sequence_key = [x for x in list(align_dict.keys()) if x not in frp_key]
            mismatch_dict = defaultdict(dict)
            sequence_dict = defaultdict(dict)

            for primer in frp_key:
                for key in sequence_key:
                    key_list = list(align_dict[key])
                    primer_boolean_list = [x != '-' for x in align_dict[primer]] 
                    filtered_sequence = ''.join([i for (i, v) in zip(key_list, primer_boolean_list) if v])
                    primer_sequence = str(align_dict[primer]).replace("-", "")
                    mismatch_dict[primer][key] = [i+1 for i in range(0, len(primer_sequence)) \
                                                  if filtered_sequence[i].lower() not in degenerate_base(primer_sequence[i]).lower()]
                    sequence_dict[primer][key] = filtered_sequence

            df = pd.DataFrame(mismatch_dict)
            for col in df.columns:
                df[col + '_num'] = df[col].str.len()

            df2 = pd.DataFrame(sequence_dict)
            df2.columns = [x + '_sequence' for x in df2.columns]

        dlist.append(df)
        dlist2.append(df2)

    df3 = pd.concat([pd.concat(dlist), pd.concat(dlist2)], axis=1)
    return df3

In [10]:
path = '/run/user/1701730025/gvfs/smb-share:server=s455fm120,share=share/DIS/LABS/Environmental/WNV/alignment/ZIKV/'
db = align_summary(path)
db.to_csv(path + 'ZIKV_alignment_2.csv', index = True)

In [11]:
# Run this cell if you only have one set of primers and probe
df = pd.DataFrame(mismatch_dict)
df['mismatch_forward'] = df[is_frp('forward', frp_key)].str.len()
df['mismatch_reverse'] = df[is_frp('reverse', frp_key)].str.len()
df['mismatch_probe'] = df[is_frp('probe', frp_key)].str.len()
df['mismatch_total'] = df['mismatch_forward'] + df['mismatch_reverse'] + df['mismatch_probe']
df.sort_values(by = ['mismatch_total'], ascending=False)

NameError: name 'mismatch_dict' is not defined

In [17]:
db

,ZIKE_forward,ZIKNS_probe,ZIKE_reverse,ZIKE_probe,ZIKNS_reverse,ZIKNS_forward,ZIKE_forward_num,ZIKNS_probe_num,ZIKE_reverse_num,ZIKE_probe_num,ZIKNS_reverse_num,ZIKNS_forward_num,ZIKE_forward_sequence,ZIKNS_probe_sequence,ZIKE_reverse_sequence,ZIKE_probe_sequence,ZIKNS_reverse_sequence,ZIKNS_forward_sequence
KX830960.1,"[7, 9, 12, 15, 21]","[3, 6, 9, 15, 21]","[11, 23]","[15, 18, 21]",[],[17],5,5,2,3,0,1,aagtttacgtgttctaagaagat,tttcgagcaaaagacggctgctggt,atctggagtatcggataatgcta,accgggaagagcattcaaccgga,tggaatggagataaggccc,gcacaatgcccccactat
KX377335.1,"[7, 9, 12, 15, 21]","[3, 6, 9, 15, 21]","[11, 23]","[15, 18, 21]",[],[17],5,5,2,3,0,1,aagtttacgtgttctaagaagat,tttcgagcaaaagacggctgctggt,atctggagtatcggataatgcta,accgggaagagcattcaaccgga,tggaatggagataaggccc,gcacaatgcccccactat
LC002520.1,"[7, 9, 12, 15, 21]","[3, 6, 9, 15, 21]","[11, 23]","[15, 18, 21]",[],[17],5,5,2,3,0,1,aagtttacgtgttctaagaagat,tttcgagcaaaagacggctgctggt,atctggagtatcggataatgcta,accgggaagagcattcaaccgga,tggaatggagataaggccc,gcacaatgcccccactat
KU955595.1,"[7, 9]","[9, 15, 21]","[2, 3, 23]","[3, 6, 21]",[],"[11, 17]",2,3,3,3,0,2,aagtttacgtgctccaagaaaat,ttccgggcgaaagacggctgctggt,acttggagtaccggataatgcta,acaggcaagagcatccagccgga,tggaatggagataaggccc,gcacaatgcctccactat
KU955592.1,"[7, 9]","[9, 15, 21]","[2, 3, 23]","[3, 6, 21]",[],"[11, 17]",2,3,3,3,0,2,aagtttacgtgctccaagaaaat,ttccgggcgaaagacggctgctggt,acttggagtaccggataatgcta,acaggcaagagcatccagccgga,tggaatggagataaggccc,gcacaatgcctccactat
KU955591.1,"[7, 9]","[9, 15, 21]","[2, 3, 23]","[3, 6, 21]",[],"[11, 17]",2,3,3,3,0,2,aagtttacgtgctccaagaaaat,ttccgggcgaaagacggctgctggt,acttggagtaccggataatgcta,acaggcaagagcatccagccgga,tggaatggagataaggccc,gcacaatgcctccactat
KX197205.1,[],[],[],[],[],[],0,0,0,0,0,0,aagtttgcatgctccaagaaaat,ttccgggctaaagatggctgttggt,atctggagtaccggataatgctg,accgggaagagcatccagccaga,tggaatggagataaggccc,gcacaatgcccccactgt
KJ776791.2,[],[],[],[],[],[],0,0,0,0,0,0,aagtttgcatgctccaagaaaat,ttccgggctaaagatggctgttggt,atctggagtaccggataatgctg,accgggaagagcatccagccaga,tggaatggagataaggccc,gcacaatgcccccactgt
KX377337.1,[],[],[],[],[],[],0,0,0,0,0,0,aagtttgcatgctccaagaaaat,ttccgggctaaagatggctgttggt,atctggagtaccggataatgctg,accgggaagagcatccagccaga,tggaatggagataaggccc,gcacaatgcccccactgt
KX262887.1,[],[],[],[],[],[],0,0,0,0,0,0,aagtttgcatgctccaagaaaat,ttccgggctaaagatggctgttggt,atctggagtaccggataatgctg,accgggaagagcatccagccaga,tggaatggagataaggccc,gcacaatgcccccactgt


In [18]:
bampath = "/run/user/1701730025/gvfs/smb-share:server=s455fm120,share=share/DIS/LABS/Environmental/WNV/fasta_sequence/WNV_BAM/"
samfile = pysam.AlignmentFile(bampath + "SRR7784795_align_dedup_sort.bam", "rb")

In [10]:
samfile = pysam.AlignmentFile(bampath + "SRR7784795_align_dedup_sort.bam", "rb" )
xx = 0
for pileupcolumn in samfile.pileup("NC_009942.1", 1, 500):
    xx = xx + 1
    for pileupread in pileupcolumn.pileups:
        if not pileupread.is_del and not pileupread.is_refskip and pileupcolumn.pos in indices:
                # query position is None if is_del or is_refskip is set.
                print ('\tbase in read %s = %s' %
                    (pileupread.alignment.query_name,
                    pileupread.alignment.query_sequence[pileupread.query_position]))
samfile.close()

NameError: name 'indices' is not defined

In [614]:
poslist = []
countlist = []
base_list = []
unique_base_count = []

samfile = pysam.AlignmentFile(bampath + "SRR7784795_align_dedup_sort.bam", "rb" )
for pileupcolumn in samfile.pileup("NC_009942.1", 1, 11029):
    poslist.append(pileupcolumn.pos)
    countlist.append(pileupcolumn.n)
    
    templist = []
    for pileupread in pileupcolumn.pileups:
        if not pileupread.is_del and not pileupread.is_refskip:
            templist.append(pileupread.alignment.query_sequence[pileupread.query_position])
    base_list.append(templist)
    unique_base_count.append(len(set(templist)))    

In [567]:
indices = [i for i, x in enumerate(unique_base_count) if x == 3]

In [568]:
print(indices)

[186, 187, 201, 213, 216, 218, 221, 222, 238, 244, 250, 258, 268, 269, 288, 292, 295, 299, 302, 303, 313, 324, 330, 331, 332, 349, 353, 363, 367, 382, 390, 396, 409, 413, 416, 426, 433, 450, 456, 467, 488, 489, 490, 505, 511, 512, 515, 536, 538, 541, 543, 554, 555, 556, 559, 566, 571, 583, 592, 597, 598, 599, 601, 608, 618, 624, 630, 636, 648, 670, 678, 697, 702, 705, 709, 711, 718, 724, 725, 726, 732, 737, 745, 750, 752, 757, 758, 764, 767, 775, 776, 779, 781, 782, 786, 790, 792, 794, 796, 798, 800, 809, 811, 812, 814, 815, 816, 821, 827, 834, 837, 843, 850, 852, 855, 856, 857, 864, 868, 871, 873, 874, 883, 887, 888, 890, 895, 896, 899, 907, 908, 910, 911, 913, 917, 918, 920, 921, 922, 923, 925, 927, 931, 932, 933, 934, 935, 938, 944, 949, 969, 978, 980, 987, 988, 989, 990, 992, 994, 996, 997, 999, 1000, 1002, 1007, 1010, 1011, 1017, 1022, 1033, 1036, 1040, 1048, 1055, 1058, 1063, 1066, 1075, 1078, 1081, 1093, 1108, 1129, 1140, 1171, 1178, 1195, 1205, 1234]


In [569]:
from collections import Counter

bool_list = []
for i in indices: 
    l =  base_list[i]
    c = Counter(l)
    if max([(c[i] / len(l)) for i in c]) < 0.98:
        bool_list.append(i)

In [574]:
bool_list

[186, 213, 1234]

In [608]:
samfile = pysam.AlignmentFile(bampath + "SRR7784795_align_dedup_sort.bam", "rb" )
pos = []
gap_delete = []
for pileupcolumn in samfile.pileup("NC_009942.1", 1, 11029):
    pos.append(pileupcolumn.n)
    if pileupcolumn.n >= 1:
        gap_delete.append(pileupcolumn.pos)